In [1]:
import os
import pandas as pd
import re
import numpy as np
import datetime

In [2]:
def get_sku_sales_date(xls_file_name):
    date_pattern = r"\d{4}-\d{2}-\d{2}"
    sku_date = re.findall(date_pattern,xls_file_name)[0]
    return sku_date

In [3]:
itemList = ["塞德思","灵的融","希纳露","百热朗","健眠计划","棒维素","琵力消","整肠药"]
def tag_add(itemName,itemLists):
    for item in itemLists:
        if item in itemName:
            return item
        else:
            continue
    return itemName

In [4]:
xls_list=os.listdir(r"D:\盐野义\数据\taoke")
bilibili_xls_list = [x for x in xls_list if "定向计划" in x or "商品维度" in x]
taoke_xls_list = [x for x in xls_list if "团长活动" in x or "自选计划" in x  or "通用计划" in x]

In [5]:
bilibili_data = pd.read_csv("D:/盐野义/数据/taoke/"+bilibili_xls_list[0])
for bilibili_csv in bilibili_xls_list[1:]:
    bilibili_data_sub = pd.read_csv("D:/盐野义/数据/taoke/"+bilibili_csv)
    bilibili_data = pd.concat([bilibili_data,bilibili_data_sub],axis=0,ignore_index=True)

In [6]:
bilibili_data = pd.read_csv("D:/盐野义/数据/taoke/"+bilibili_xls_list[0])
bilibili_data["日期"] = pd.to_datetime(get_sku_sales_date(bilibili_xls_list[0]))
for bilibili_csv in bilibili_xls_list[1:]:
    bilibili_data_sub = pd.read_csv("D:/盐野义/数据/taoke/"+bilibili_csv)
    bilibili_data_sub["日期"] = pd.to_datetime(get_sku_sales_date(bilibili_csv))
    bilibili_data = pd.concat([bilibili_data,bilibili_data_sub],axis=0,ignore_index=True)
bilibili_data["商品名称"] = bilibili_data["商品名称"].fillna("其他")
bilibili_data["商品别名"] = bilibili_data["商品名称"].apply(lambda x:tag_add(x,itemList))

In [7]:
taoke_data = pd.read_csv("D:/盐野义/数据/taoke/"+taoke_xls_list[0])
taoke_data["日期"] = pd.to_datetime(get_sku_sales_date(taoke_xls_list[0]))
for taoke_csv in taoke_xls_list[1:]:
    taoke_data_sub = pd.read_csv("D:/盐野义/数据/taoke/"+taoke_csv)
    taoke_data_sub["日期"] = pd.to_datetime(get_sku_sales_date(taoke_csv))
    taoke_data = pd.concat([taoke_data,taoke_data_sub],axis=0,ignore_index=True)
taoke_data["商品名称"] = taoke_data["商品名称"].fillna("其他")
taoke_data["商品别名"] = taoke_data["商品名称"].apply(lambda x:tag_add(x,itemList))

In [8]:
bilibili_pivot = bilibili_data.pivot_table(values="付款人数",index="商品别名",columns="日期",aggfunc="sum")
taoke_pivot = taoke_data.pivot_table(values="付款人数",index="商品别名",columns="日期",aggfunc="sum")

In [9]:
bilibili_pivot.to_excel("D:/盐野义/数据/每周淘客/bilibili推广_"+str(datetime.date.today())+".xlsx")
taoke_pivot.to_excel("D:/盐野义/数据/每周淘客/taoke推广_"+str(datetime.date.today())+".xlsx")